# Coding Assignment 2

#### by Arnav Das (arnavmd2@uw.edu)

# Introduction to the Assignment  

In this assignment, you will build a model to **classify the sentiment of Yelp reviews**.

The **Yelp Reviews** dataset is a large collection of restaurant and business reviews written by users and annotated with **sentiment labels**.  
Each example contains a piece of review text along with a label indicating how positive or negative the reviewer’s experience was.

The goal is to **understand and categorize the sentiment expressed in each review** based solely on its text.

The labels correspond to different levels of sentiment:

#### 1-star — Strongly negative  
#### 2-star — Negative  
#### 3-star — Neutral  
#### 4-star — Positive  
#### 5-star — Strongly positive  

This dataset is widely used in **text classification and sentiment analysis**, making it a strong benchmark for evaluating how well models can interpret opinions, tone, and emotional content in written language.

# Part 1: Dataset Exploration (10 points)

Here we will do some quick sanity checks on the dataset, before we do any sort of training.

In [ ]:
from datasets import load_dataset

SEED = 42
N_TRAIN = 50_000
N_VAL   = 5_000
N_EVAL  = 500

ds = load_dataset("yelp_review_full")
train_full = ds["train"].shuffle(seed=SEED)
train = train_full.select(range(N_TRAIN))
val   = train_full.select(range(N_TRAIN, N_TRAIN + N_VAL))

test = ds["test"].shuffle(seed=SEED).select(range(N_EVAL))

In [ ]:
#TODO Put your code here (2.5 pts).
###########################################################################
# Plot a bar graph, that shows how many samples there are each class

###########################################################################

In [ ]:
import re
from collections import Counter
from tqdm import tqdm

NUM_CLASSES = 5
N = 5  # Try a few different values for this
MAX_SAMPLES = 10_000 # We only look at 10_000 samples for speed

# Each class label (0–4) gets its own Counter
ngram_counts = {i: Counter() for i in range(NUM_CLASSES)}


# Low level tokens like the ones BERT uses are not that useful, so we will construct coarser tokens for analysis
def simple_tokenize(text):
    text = text.lower()
    text = text.replace("\n", " ")
    text = re.sub(r"[^a-z0-9' ]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split(" ")


for i, example in tqdm(enumerate(train)):
    if MAX_SAMPLES and i >= MAX_SAMPLES:
        break

    text = example["text"]
    label = example["label"]

    tokens = simple_tokenize(text)
    pass
    #TODO: Put your code here (2.5 pts).
    ###########################################################################
    #   Build all N-grams from `tokens` and count them
    #   in `ngram_counts[label]`
    #
    #   An N-gram is a tuple of N consecutive tokens.
    #
    #   Example (N=3):
    #     tokens = ["this","place","is","really","good"]
    #     trigrams = [
    #        ("this","place","is"),
    #        ("place","is","really"),
    #        ("is","really","good")
    #     ]
    # --------------------------------------------------
    ###########################################################################



# Extract top 5 n-grams per class
top_ngrams = {i: ngram_counts[i].most_common(5) for i in range(NUM_CLASSES)}

star_names = {
    0: "1-star",
    1: "2-star",
    2: "3-star",
    3: "4-star",
    4: "5-star",
}

for label in range(NUM_CLASSES):
    print("\n", star_names[label])
    for (ngram, count) in top_ngrams[label]:
        print(f"{' '.join(ngram):30s}  {count}")


### TODO:
Please fill out the following questions:


**Q1 [2.5 points]: Is the dataset balanced?** [Your Response]

**Q2 [2.5 points]: Try a few different values for n and print out the top 5 n-grams for for each class. Which value of n gives you n-grams that are strongly tied to sentiment (e.g., “highly recommend this”, “worst place ever”)** [Your Response]

# Part 2: GPT 4.1-Nano Zero-Shot Performance (20 points)

In this section, we evaluate GPT-4.1-nano as a zero-shot text classifier.
Zero-shot means the model is not trained on our dataset. Instead, we give it instructions in natural language and ask it to predict a label directly.

We will provide GPT with a prompt that describes the task (e.g., predicting a 1–5 star rating from a review) and pass in each test example. The model should output a single label based only on its pretrained knowledge.



In [ ]:
import os, json, time, re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from datasets import load_dataset
from sklearn.metrics import confusion_matrix, accuracy_score

from openai import OpenAI


MODEL = "gpt-4.1-nano"
TEMPERATURE = 0.0
SLEEP_S = 0.0  # set to 0.05 if you hit rate limits
NUM_RETRIES = 5

API_KEY= ""
client = OpenAI(api_key=API_KEY)


def gpt_predict_label(review_text: str) -> int:
    #TODO: Write a prompt that asks GPT to label a sample (5 points)
    ###########################################################################
    system = ""
    user = ""
    ###########################################################################

    resp = client.responses.create(
        model=MODEL,
        input=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
        temperature=TEMPERATURE,
    )

    raw = resp.output_text  # plain text output
    return extract_label(raw)

def extract_label(text: str) -> int:
    """
    Robustly extract a label in {0,1,2,3,4} from model output.
    """
    #TODO: Put your code here (5 points).
    ###########################################################################

    ###########################################################################
    return 0

In [ ]:
# Get predictions for the full test set
y_true, y_pred = [], []

for ex in tqdm(test, desc=f"GPT inference: {MODEL}"):
    y_true.append(int(ex["label"]))

    retries = 0
    while retries < 5:
        try:
            y_pred.append(gpt_predict_label(ex["text"]))
            break
        except Exception as e:
            print("Failed to generate response", e)
            print("Retrying...")
            if SLEEP_S:
                time.sleep(SLEEP_S)

        if SLEEP_S:
            time.sleep(SLEEP_S)

acc = accuracy_score(y_true, y_pred)
print(f"{MODEL} | N={N_EVAL} | accuracy={acc:.4f}")

cm = confusion_matrix(y_true, y_pred, labels=[0,1,2,3,4])

plt.figure(figsize=(6,5))
sns.heatmap(
    cm, annot=True, fmt="d",
    xticklabels=["1★","2★","3★","4★","5★"],
    yticklabels=["1★","2★","3★","4★","5★"],
)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title(f"Confusion Matrix — {MODEL} (Yelp)")
plt.show()

### TODO:
Please fill out the following questions:


**Q3 (2.5 points): What accuracy does GPT attain on this dataset?** [Your Response]

**Q4 (2.5 points): What are the biggest failure modes of GPT?** [Your Response]

**Q5 (5 points): What are some ways to improve GPT performance on this dataset?** [Your Response]

# Part 3: BERT-Based Classification (50 points)

Now we use BERT as a feature extractor for classification.
We keep BERT frozen and train a small neural network on top of it to predict the labels.

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer
from torch.optim import AdamW
from datasets import load_dataset
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# Function for tokenizing text
def manual_encode(text, max_length=512):
    """
    Encode a single Yelp review for BERT.
    Format: [CLS] text [SEP]
    """

    # Tokenize the text, and pad to max_length for collation
    tokens = tokenizer.tokenize(text)
    max_text_len = max_length - 2
    tokens = tokens[:max_text_len]
    tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    attention_mask = [1] * len(input_ids)

    # Pad to max_length
    padding_length = max_length - len(input_ids)
    input_ids += [tokenizer.pad_token_id] * padding_length
    attention_mask += [0] * padding_length

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }


In [ ]:
def update_dataset(example):
    return manual_encode(example["text"])

train = train.map(update_dataset)
val  = val.map(update_dataset)
test  = test.map(update_dataset)


train = train.remove_columns(["text"])
val = val.remove_columns(["text"])
test  = test.remove_columns(["text"])

train.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

train_loader = DataLoader(train, batch_size=32, shuffle=True)
val_loader   = DataLoader(val, batch_size=32)
test_loader  = DataLoader(test, batch_size=32)

In [ ]:
import copy
import torch

@torch.no_grad()
def compute_embeddings(bert_model, loader, device, context_rep="cls"):
    bert_model.eval()
    all_embs = []
    all_labels = []

    for batch in tqdm(loader, desc=f"Embedding ({context_rep})"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden = outputs.last_hidden_state  # (B, T, H)

        if context_rep == "cls":
            #TODO Put your code here (5 points).
            ###########################################################################
            # last_hidden_state: (B, T, H); take [CLS] token at position 0
            ###########################################################################
            pass
        elif context_rep == "mean_pool":
            #TODO Put your code here (5 points).
            ###########################################################################
            # Mean pooling over tokens, ignoring padding via attention_mask

            ###########################################################################
            pass
        else:
            raise ValueError(f"Unknown context_rep: {context_rep}")

        all_embs.append(emb.cpu())
        all_labels.append(labels.cpu())

    X = torch.cat(all_embs, dim=0)      # (N, H)
    y = torch.cat(all_labels, dim=0)    # (N,)
    return X, y


class EmbeddingDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


def make_head(hidden_size, num_labels=5, classifier_type="linear"):
    if classifier_type == "linear":
        #TODO: Put your code here (5 points).
        # Add a linear layer which maps the pooled output of BERT to the number of desired classes (num_labels).
        # The linear layer essentially acts as a classifier on top of the BERT model, using the pooled representation of the input sequence to make class predictions.
        ###########################################################################
        clf = None
        ###########################################################################
    elif classifier_type == "non-linear":
        #TODO: Put your code here (5 points).
        # Add two linear layers which map the pooled output of BERT to the number of desired classes (num_labels).
        # The first linear layer should be of size [hidden_size, 0.5*hidden_size]
        # The second linear layer should be of size [0.5*hidden_size, num_labels]
        # Add a RELU function between these two layers
        ###########################################################################
        clf = None
        ###########################################################################
    else:
        raise ValueError(f"Unknown classifier_type: {classifier_type}")


    return clf

def train_head(
    head,
    train_loader,
    val_loader,
    optimizer,
    device,
    num_epochs=10,
    eval_every_epochs=1,
    eval_on_last_epoch=True,
):
    """
    Trains `head` and keeps the best-performing weights (by val_loss) in memory.
    Returns the head loaded with the best weights.

    Evaluation happens every `eval_every_epochs` epochs (at the end of the epoch),
    and optionally on the final epoch as well.
    """
    loss_fn = nn.CrossEntropyLoss()
    best_val_loss = float("inf")
    best_state = None  # in-memory copy of best weights

    def run_validation():
        head.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for Xv, yv in val_loader:
                #TODO: Put your code here (5 points).
                # This code is to loop over all validation set embeddings and compute the loss and accuracy
                ###########################################################################

                ###########################################################################
                pass
        val_acc = correct/total

        return val_loss, val_acc


    for epoch in range(num_epochs):
        head.train()
        for X, y in tqdm(train_loader, desc=f"Head epoch {epoch+1}/{num_epochs}"):
            #TODO: Put your code here (10 points)
            # Add training logic here
            ###########################################################################

            ###########################################################################
            pass

        should_eval = ((epoch + 1) % eval_every_epochs == 0) or (
            eval_on_last_epoch and (epoch + 1 == num_epochs)
        )
        if should_eval:
            val_loss, val_acc = run_validation()
            print(f"[epoch {epoch+1}] val_loss={val_loss:.4f} val_acc={val_acc:.4f}")

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_state = copy.deepcopy(head.state_dict())
                print("  -> new best head kept in memory")

    # Load best weights back into `head` before returning
    if best_state is not None:
        head.load_state_dict(best_state)

    return head

In [ ]:
# Function for evaluation and plotting confusion matrix (HEAD ONLY, EMBEDDING BATCHES ONLY)
def evaluate_and_plot(head, test_loader, device, classifier_type, context_rep):
    """
    Assumes:
      - test_loader yields batches of (X, y) from EmbeddingDataset
      - head(X) returns logits of shape (B, num_classes)
    """
    head.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for X, y in test_loader:
            X = X.to(device)
            y = y.to(device)

            logits = head(X)
            preds = logits.argmax(dim=1)

            y_true.append(y.detach().cpu().numpy())
            y_pred.append(preds.detach().cpu().numpy())

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(6,5))
    sns.heatmap(
        cm, annot=True, fmt="d",
        xticklabels=["1★","2★","3★","4★","5★"],
        yticklabels=["1★","2★","3★","4★","5★"],
    )
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(f"Confusion Matrix — {classifier_type} ({context_rep})")
    plt.show()

    accuracy = float((y_true == y_pred).mean()) if len(y_true) > 0 else 0.0
    print(f"classifier_type-{classifier_type} context_rep-{context_rep}")
    print(f"Accuracy: {accuracy:.4f}")

    return accuracy, cm

In [ ]:
# Hyperparameters
CONTEXT_REP = 'cls'
CLF_TYPE = 'non-linear'
LEARNING_RATE = 0.1
BATCH_SIZE = 32

# 1) Load BERT once and freeze it
device = 'cuda:0'
bert = BertModel.from_pretrained("bert-base-uncased").to(device)
for p in bert.parameters():
    p.requires_grad_(False)

# 2) Precompute embeddings ONCE
X_train, y_train = compute_embeddings(bert, train_loader, device, context_rep=CONTEXT_REP)
X_val, y_val     = compute_embeddings(bert, val_loader, device, context_rep=CONTEXT_REP)
X_test, y_test   = compute_embeddings(bert, test_loader, device, context_rep=CONTEXT_REP)

# 3) Build embedding loaders (much faster)
train_emb_loader = DataLoader(EmbeddingDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
val_emb_loader   = DataLoader(EmbeddingDataset(X_val, y_val), batch_size=BATCH_SIZE, shuffle=False)
test_emb_loader  = DataLoader(EmbeddingDataset(X_test, y_test), batch_size=BATCH_SIZE, shuffle=False)

# 4) Train a small head for many epochs cheaply
hidden_size = X_train.shape[1]
head = make_head(hidden_size, num_labels=5, classifier_type=CLF_TYPE).to(device)
opt = torch.optim.AdamW(head.parameters(), lr=LEARNING_RATE)

best_head = train_head(head, train_emb_loader, val_emb_loader, opt, device, num_epochs=10, eval_every_epochs=1)

In [ ]:
ret = evaluate_and_plot(best_head, test_emb_loader, device, classifier_type=CLF_TYPE, context_rep=CONTEXT_REP)

### TODO:
Please fill out the following questions:


**Q6 (5 points): Test all configurations of BERT (both head types and context representation types) and fill out a table of results. For each configuration, find a learning rate that gives reasonable performance (you do not need to report every learning rate tried).** [Your Response]

**Q7 (5 points): Aside from any accuracy gains you may have observed, what are two key advantages of using a BERT-based classifier over using GPT?** [Your Response]

**Q8 (5 points): What are two advantages of using GPT instead of the BERT-based classifier?** [Your Response]

## **Part 4: Bakeoff (20 points)**

In this final section, you will take part in a friendly competition. You may use **any classifier of your choice** to generate predictions for a set of unlabeled examples.

Download the data from the Kaggle link:  https://www.kaggle.com/t/6f431e5fab1648e4a8af869a44405984

You are given two CSV files:

- `text.csv` — Contains two columns: an ID and the corresponding raw text. You must generate a prediction for each row.
- `sample_submission.csv` — Shows the required format for your submission file.

Once you have generated your predictions, submit them to the Kaggle competition using the above link.

In [1]:
#TODO: Put your code here (5 points)


**Q9 (5 points): Write a detailed description of your chosen approach and explain any changes you made.** [Your Response]

**Q10 (10 points): Submit your predictions on Kaggle in the link we provided. You will receive full points for a valid submission.**